In [1]:
import os
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
import psycopg2

In [5]:
with psycopg2.connect("host=localhost port=5432 dbname=Splatoon user=tidal password=tidalryoku") as conn:
    with conn.cursor() as cur:
        #train data取得
        cur.execute('SELECT * FROM train')
        rows = cur.fetchall()
        train = pd.DataFrame(rows)
        colnames = [col.name for col in cur.description]
        train.columns = colnames
        
        #test data取得
        cur.execute('SELECT * FROM test')
        rows = cur.fetchall()
        test = pd.DataFrame(rows)
        colnames = [col.name for col in cur.description]
        test.columns = colnames

In [8]:
train

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour,y
0,1,regular,nawabari,sumeshi,sshooter_becchu,None,139,soytuber_custom,None,118,...,None,68,sharp_neo,None,31,2019-10-15,20:00:00,78,20,1
1,2,regular,nawabari,arowana,parashelter_sorella,None,198,jetsweeper,None,77,...,None,168,sputtery_clear,None,151,2019-12-14,04:00:00,18,4,0
2,3,gachi,hoko,ama,nzap89,a-,114,quadhopper_black,a,68,...,a-,160,dualsweeper_custom,a,126,2019-12-25,14:00:00,7,14,0
3,4,regular,nawabari,engawa,bamboo14mk1,None,336,splatroller_becchu,None,131,...,None,194,hotblaster_custom,None,391,2019-11-11,14:00:00,51,14,0
4,5,gachi,hoko,chozame,bold_7,x,299,hissen_hue,x,97,...,x,246,wakaba,x,160,2019-12-14,06:00:00,18,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,66121,gachi,area,devon,hokusai_becchu,a,66,soytuber,a,113,...,a,38,wakaba,a+,283,2019-10-11,10:00:00,82,10,1
66121,66122,gachi,asari,otoro,hokusai_becchu,s+,204,prime_becchu,s+,96,...,s+,188,nzap89,s+,170,2019-12-04,00:00:00,28,0,1
66122,66123,gachi,hoko,bbass,prime_collabo,a+,116,promodeler_rg,a,86,...,a+,191,hissen_hue,a+,139,2019-10-22,00:00:00,71,0,1
66123,66124,gachi,area,fujitsubo,heroroller_replica,s+,210,sharp_neo,s+,96,...,s+,223,sharp_neo,s+,194,2019-12-11,00:00:00,21,0,0


In [9]:
test

,id,lobby_mode,mode,stage,a1_weapon,a1_rank,a1_level,a2_weapon,a2_rank,a2_level,...,b3_weapon,b3_rank,b3_level,b4_weapon,b4_rank,b4_level,yyyymmdd,time,datedelta,hour
0,1,gachi,area,hakofugu,prime_collabo,x,174,herospinner_replica,x,130,...,l3reelgun_d,x,213,nzap89,x,306,2019-12-17,12:00:00,15,12
1,2,gachi,asari,anchovy,prime_becchu,s+,363,nzap89,s+,59,...,prime_becchu,s+,147,splatroller,s+,156,2019-11-25,16:00:00,37,16
2,3,gachi,area,mutsugoro,furo_deco,s,116,bold,s,125,...,nzap83,s,183,promodeler_pg,s,105,2019-10-22,08:00:00,71,8
3,4,gachi,asari,devon,prime_becchu,s+,192,splatspinner_collabo,s+,115,...,soytuber,s+,172,prime,s+,70,2019-12-30,04:00:00,2,4
4,5,gachi,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,x,223,...,splatscope,x,50,screwslosher_becchu,x,287,2019-10-15,16:00:00,78,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28335,28336,gachi,yagura,battera,prime_becchu,x,67,splatroller,x,62,...,prime_becchu,x,240,nzap83,x,150,2019-10-11,16:00:00,82,16
28336,28337,gachi,yagura,anchovy,longblaster_necro,c-,25,splatroller_collabo,c-,18,...,nzap89,c-,17,dualsweeper,c+,35,2019-10-20,12:00:00,73,12
28337,28338,gachi,yagura,zatou,furo,x,386,kugelschreiber,x,136,...,carbon_deco,x,257,l3reelgun_d,x,243,2019-12-14,00:00:00,18,0
28338,28339,gachi,asari,hokke,maneuver_becchu,s+,185,wakaba,s+,103,...,hokusai,s+,152,ochiba,s+,130,2019-10-12,04:00:00,81,4


In [11]:
#ラベルエンコード(文字列→数値)
def Label_encode(train, test, feature_name):
    for f in feature_name:
        lbl = LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))
    
    return train, test

## 欠損値補完

In [18]:
def FillnaAndInsertIsnan(DataFrame, ColsAndFillVals):
    for (col, val) in ColsAndFillVals:
        IsnanSeries = DataFrame[col].isnull()
        DataFrame[col] = DataFrame[col].fillna(val)
        DataFrame.insert(len(DataFrame.columns), col + "_isnan", IsnanSeries)
    return DataFrame

In [19]:
train.columns

Index(['id', 'lobby_mode', 'mode', 'stage', 'a1_weapon', 'a1_rank', 'a1_level',
       'a2_weapon', 'a2_rank', 'a2_level', 'a3_weapon', 'a3_rank', 'a3_level',
       'a4_weapon', 'a4_rank', 'a4_level', 'b1_weapon', 'b1_rank', 'b1_level',
       'b2_weapon', 'b2_rank', 'b2_level', 'b3_weapon', 'b3_rank', 'b3_level',
       'b4_weapon', 'b4_rank', 'b4_level', 'yyyymmdd', 'time', 'datedelta',
       'hour', 'y'],
      dtype='object')

In [ ]:
mean_A4level = train['a4_level'].mean()
mean_B3level = train['b3_level'].mean()
mean_B4level = train['b4_level'].mean()
train = FillnaAndInsertIsnan(train, 
                             [('a1_rank','na'), ('a2_rank','na'), ('a3_rank','na'), ('A4-rank','na'), ('A4-weapon','na'), 
                              ('A4-level',mean_A4level), ('B1-rank','na'), ('B2-rank','na'), ('B3-rank','na'), ('B4-rank','na'),
                              ('B3-weapon','na'), ('B3-level',mean_B3level), ('B4-weapon','na'), ('B4-level',mean_B4level)])
test = FillnaAndInsertIsnan(test, 
                             [('A1-rank','na'), ('A2-rank','na'), ('A3-rank','na'), ('A4-rank','na'), ('A4-weapon','na'), 
                              ('A4-level',mean_A4level), ('B1-rank','na'), ('B2-rank','na'), ('B3-rank','na'), ('B4-rank','na'),
                              ('B3-weapon','na'), ('B3-level',mean_B3level), ('B4-weapon','na'), ('B4-level',mean_B4level)])